In [9]:
import pandas as pd
import numpy as np
import re
import nltk
import pymorphy2
import patoolib
import os
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from transformers import BertTokenizer, BertModel
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GroupShuffleSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score, precision_recall_curve, classification_report, accuracy_score
from catboost import CatBoostClassifier, Pool
from tqdm import tqdm
from bs4 import BeautifulSoup
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import StackingClassifier
import warnings
from sklearn.cluster import DBSCAN
warnings.filterwarnings("ignore")

/Users/taniyashuba/PycharmProjects/VK_ML_Project/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Загрузка данных

In [10]:
# Загрузка данных
train_groups = pd.read_csv('train_groups.csv')
test_groups = pd.read_csv('test_groups.csv')
sample_submission = pd.read_csv('sample_submission.csv')
docs_titles = pd.read_csv('docs_titles.tsv', sep='\t')

In [11]:
# Объединение заголовков с данными групп
train_data = train_groups.merge(docs_titles, on='doc_id')
test_data = test_groups.merge(docs_titles, on='doc_id', how='left')

In [12]:
# Загрузка данных из дополнительного файла
additional_data = pd.read_csv('doc_texts_processed.tsv', sep='\t')

In [13]:
# Объединение дополнительного файла с основными данными
train_data = train_data.merge(additional_data, on='doc_id', how='left', suffixes=('', '_add'))
test_data = test_data.merge(additional_data, on='doc_id', how='left', suffixes=('', '_add'))

In [14]:
# Проверка наличия дополнительного столбца и его обработка
if 'text_processed_add' in train_data.columns:
    train_data['title'].fillna(train_data['text_processed_add'], inplace=True)
if 'text_processed_add' in test_data.columns:
    test_data['title'].fillna(test_data['text_processed_add'], inplace=True)

In [15]:
# Просмотр пропусков
train_data.isna().sum(), test_data.isna().sum()

(pair_id            0
 group_id           0
 doc_id             0
 target             0
 title             16
 text_processed     0
 dtype: int64,
 pair_id               0
 group_id              0
 doc_id                0
 title                92
 text_processed    16485
 dtype: int64)

In [16]:
# Заполнение оставшихся пропусков пустыми строками
train_data['title'].fillna('', inplace=True)
test_data['title'].fillna('', inplace=True)

# Предобработка данных


In [17]:
# Загрузка данных для NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/taniyashuba/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/taniyashuba/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/taniyashuba/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
# Стоп-слова и шум
stop_words = set(stopwords.words('russian')) | set(stopwords.words('english'))

In [19]:
# Пробуем стеммер или леммер
stemmer = SnowballStemmer("russian")
morph = pymorphy2.MorphAnalyzer()


def preprocess_text(text):
    """
    Токенизация + лемматизация/стемминг текста
    """
    text = re.sub(r'<.*?>', '', text)  # Удаление HTML-тегов
    text = re.sub(r'[^a-zA-Zа-яА-Я0-9\s]', '', text.lower())  # Удаление спецсимволов
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words and not token.isdigit()]
    tokens = [morph.parse(word)[0].normal_form for word in tokens]
    return tokens


train_data['title_processed'] = train_data['title'].apply(preprocess_text)
test_data['title_processed'] = test_data['title'].apply(preprocess_text)

# Векторизация BERT

In [20]:
# Векторизация с помощью BERT
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)


def get_bert_embeddings(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().cpu()


train_data['title_embeddings'] = train_data['title'].apply(lambda x: get_bert_embeddings(x).numpy())
test_data['title_embeddings'] = test_data['title'].apply(lambda x: get_bert_embeddings(x).numpy())

In [21]:
# Создание фичей
def embeddings_to_features(data, column_prefix):
    """
    Преобразуем эмбеддинги в фичи
    """
    embeddings = np.stack(data[column_prefix + '_embeddings'].values)
    feature_names = [f"{column_prefix}_embedding_{i}" for i in range(embeddings.shape[1])]
    features_df = pd.DataFrame(embeddings, columns=feature_names, index=data.index)
    return features_df


train_features = embeddings_to_features(train_data, 'title')
test_features = embeddings_to_features(test_data, 'title')

In [22]:
def add_new_features(data):
    """
    Добавляем новые признаки: длина заголовка и число уникальных слов
    """
    data['title_length'] = data['title'].apply(lambda x: len(x.split()))
    data['unique_words'] = data['title_processed'].apply(lambda x: len(set(x)))
    return data


train_data = add_new_features(train_data)
test_data = add_new_features(test_data)

## Косинусные сходства Tfidf

In [23]:
def vectorize_group(group):
    """
    Векторизуем группу документов с кастомным токенайзером
    """
    vectorizer = TfidfVectorizer(tokenizer=preprocess_text)
    vectors = vectorizer.fit_transform(group['title'])
    return vectors

In [24]:
def cosine_matrix_group(group):
    """
    Вычисление матрицы косинусных расстояний для группы
    """
    tfidf_matrix = vectorize_group(group)
    cosine_matrix = cosine_similarity(tfidf_matrix)
    return cosine_matrix

In [25]:
def calc_cosine_similarity(data_grouped, count=10):
    """
    Возвращаем топ косинусных сходств для каждого документа группы
    """
    similarity_features_list = []
    for name, group in tqdm(data_grouped, desc="Processing groups"):
        cosine_matrix = cosine_matrix_group(group)
        for k, (idx, row) in enumerate(group.iterrows()):
            similarities = []
            for j in range(len(group)):
                if k == j:
                    continue
                similarities.append(cosine_matrix[k, j])
            top_similarities = sorted(similarities, reverse=True)[:count]
            similarity_record = [row['pair_id']] + top_similarities
            similarity_features_list.append(similarity_record)
    similarity_columns = ['pair_id'] + [f'top_{i + 1}_similarity' for i in range(count)]
    similarity_features = pd.DataFrame(similarity_features_list, columns=similarity_columns)
    return similarity_features

In [26]:
# Кластеризация
def clustering_features(data_grouped, eps=0.5, min_samples=5, metric='cosine'):
    """
    Возвращаем кластер для каждого документа группы
    """
    clustering_features_list = []
    for name, group in tqdm(data_grouped, desc="Processing groups"):
        cosine_matrix = cosine_matrix_group(group)
        dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric=metric).fit(cosine_matrix)
        cluster_labels = dbscan.labels_
        for k, (idx, row) in enumerate(group.iterrows()):
            clustering_record = [row['pair_id'], cluster_labels[k]]
            clustering_features_list.append(clustering_record)
    clustering_columns = ['pair_id', 'cluster']
    clustering_features = pd.DataFrame(clustering_features_list, columns=clustering_columns)
    return clustering_features

# Объединение данных и фичей в датасет


In [27]:
# Разбиваем train и test по группам на основе id
train_data_grouped = train_data.groupby('group_id')
test_data_grouped = test_data.groupby('group_id')

In [28]:
# Фичи для трейна и теста
train_similarity_features = calc_cosine_similarity(train_data_grouped)
train_clustering_features = clustering_features(train_data_grouped)

test_similarity_features = calc_cosine_similarity(test_data_grouped)
test_clustering_features = clustering_features(test_data_grouped)

Processing groups: 100%|██████████| 180/180 [00:12<00:00, 14.56it/s]


In [29]:
# Заполняем пропуски нулями
train_similarity_features = train_similarity_features.fillna(0)
test_similarity_features = test_similarity_features.fillna(0)

In [30]:
# Объединяем новые признаки с исходным датасетом
train_data = train_data.merge(train_similarity_features, on=['pair_id'])
train_data = train_data.merge(train_clustering_features, on=['pair_id'])

test_data = test_data.merge(test_similarity_features, on=['pair_id'])
test_data = test_data.merge(test_clustering_features, on=['pair_id'])

In [31]:
# Объединяем с BERT признаками
train_data = pd.concat([train_data, train_features], axis=1)
test_data = pd.concat([test_data, test_features], axis=1)

In [36]:
# Удаление ненужных столбцов, если они существуют
drop_columns = ['text_processed_add', 'title_embeddings']
train_data.drop(columns=[col for col in drop_columns if col in train_data.columns], inplace=True)
test_data.drop(columns=[col for col in drop_columns if col in test_data.columns], inplace=True)

In [37]:
# Проверка на дубликаты
train_data = train_data.drop_duplicates(subset=['doc_id', 'pair_id', 'group_id'], keep='first')

X_train = train_data.drop(columns=['doc_id', 'pair_id', 'group_id', 'target', 'title', 'title_processed'], axis=1)
y_train = train_data['target']

X_test = test_data.drop(columns=['doc_id', 'pair_id', 'group_id', 'title', 'title_processed'], axis=1)

In [38]:
print(f"Length of train_groups: {len(train_groups)}")
print(f"Length of train_data after merging with docs_titles: {len(train_data)}")
print(f"Length of train_data after merging with additional_data: {len(train_data)}")
print(f"Length of X_train: {len(X_train)}")
print(f"Length of y_train: {len(y_train)}")

Length of train_groups: 11690
Length of train_data after merging with docs_titles: 11690
Length of train_data after merging with additional_data: 11690
Length of X_train: 11690
Length of y_train: 11690


In [39]:
# Убедитесь, что все переменные имеют одинаковую длину
assert len(X_train) == len(y_train) == len(train_data)

In [40]:
# Разделение данных на обучающую и валидационную выборки
splitter = GroupShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_indices, val_indices = next(splitter.split(X_train, y_train, groups=train_data['group_id']))

X_train_split = X_train.iloc[train_indices]
y_train_split = y_train.iloc[train_indices]

X_val_split = X_train.iloc[val_indices]
y_val_split = y_train.iloc[val_indices]

# Настройка параметров базовых моделей


In [43]:
# Настройка параметров базовых моделей
knn_params = {'n_neighbors': [3, 5, 7, 10]}
knn_grid = GridSearchCV(KNeighborsClassifier(), knn_params, cv=5, scoring='f1', n_jobs=-1)
knn_grid.fit(X_train_split, y_train_split)
best_knn = knn_grid.best_estimator_

ValueError: 
All the 20 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/taniyashuba/PycharmProjects/VK_ML_Project/venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/taniyashuba/PycharmProjects/VK_ML_Project/venv/lib/python3.9/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/taniyashuba/PycharmProjects/VK_ML_Project/venv/lib/python3.9/site-packages/sklearn/neighbors/_classification.py", line 238, in fit
    return self._fit(X, y)
  File "/Users/taniyashuba/PycharmProjects/VK_ML_Project/venv/lib/python3.9/site-packa

In [ ]:
# Настройка параметров для RandomForestClassifier
rf_params = {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20, 30]}
rf_grid = GridSearchCV(RandomForestClassifier(), rf_params, cv=5, scoring='f1', n_jobs=-1)
rf_grid.fit(X_train_split, y_train_split)
best_rf = rf_grid.best_estimator_

In [ ]:
# Настройка параметров для SVC
svc_params = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
svc_grid = GridSearchCV(SVC(probability=True), svc_params, cv=5, scoring='f1', n_jobs=-1)
svc_grid.fit(X_train_split, y_train_split)
best_svc = svc_grid.best_estimator_

In [ ]:
# Настройка параметров CatBoost
cat_params = {
    'iterations': [200, 350, 500],
    'depth': [6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5]
}

cat_model = CatBoostClassifier()
cat_grid = GridSearchCV(cat_model, cat_params, cv=5, scoring='f1', n_jobs=-1)
cat_grid.fit(X_train_split, y_train_split)
best_cat_params = cat_grid.best_params_

In [ ]:
# Обучение CatBoost с лучшими параметрами на полном наборе данных
final_cat_model = CatBoostClassifier(**best_cat_params)
train_pool = Pool(X_train, y_train)
final_cat_model.fit(train_pool)

In [ ]:
# Настройка параметров мета-модели
meta_params = {'C': [0.1, 1, 10]}
meta_grid = GridSearchCV(LogisticRegression(), meta_params, cv=5, scoring='f1', n_jobs=-1)
meta_grid.fit(X_train, y_train)
best_meta = meta_grid.best_estimator_

# Стекинг


In [ ]:
# Создание стекинг-классификатора
stacking_model = StackingClassifier(
    estimators=[
        ('knn', best_knn),
        ('rf', best_rf),
        ('svc', best_svc),
        ('cat', final_cat_model)
    ],
    final_estimator=best_meta,
    cv=5
)

stacking_model.fit(X_train, y_train)


In [ ]:
# Предсказание и сохранение результатов
test_predictions = stacking_model.predict(X_test)


In [ ]:
# Сохранение результатов
submission = test_groups[['pair_id']].copy()
submission['target'] = test_predictions
submission.to_csv('submission.csv', index=False)
print('Файл с предсказаниями создан: submission.csv')


In [ ]:
# Важность признаков
import matplotlib.pyplot as plt

importances = final_cat_model.get_feature_importance(type='PredictionValuesChange')
feature_importances = pd.Series(importances, index=X_train.columns).sort_values()[-15:]
plt.figure(figsize=(10, 6))
plt.barh(feature_importances.index, feature_importances.values)
plt.title('CatBoost Feature Importance')
plt.xlabel('Importance')
plt.ylabel('Features')
plt.show()